<a href="https://colab.research.google.com/github/BamBuStiK/waldoGoogLeNet/blob/main/waldoGoogLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

성능 개선 (모델 수정, 데이터, 하이퍼메터 튜닝)

목표 80% 현재 성능 70%
1. 데이터 추가수집
2. 모델
3. 하이퍼 파라메터 수전(우선 순위 최하, 목표 80% 현재 성능 79.5%일때)

In [52]:
!pip install tensorflow

In [53]:
#from google.colab import drive
#drive.mount('/content/drive')

In [54]:
!mkdir dataset

mkdir: cannot create directory ‘dataset’: File exists


In [55]:
!unzip /content/drive/MyDrive/my_datasets.zip -d /content/dataset

Archive:  /content/drive/MyDrive/my_datasets.zip
replace /content/dataset/burglar/burglar-001 (1).png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [56]:
#waldo와 연관자들 전부 잡아내는 프로그램

In [57]:
x=[]
y=[]

In [58]:
from google.colab import drive
drive.mount('/content/drive')
#전처리
import PIL #python의 이미지 전처리 라이브러리
import glob #대용량, 대규모 파일을 다룰때 사용
import os #os, path 관련 조작, 변경, 파일 삭제 등
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.python.distribute import input_lib
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
dataset_folder_path = '/content/dataset'
category_cnt = len(os.listdir(dataset_folder_path)) #경로만 넣어주면 디렉토리 안의 파일들을 리스 형태로 보여줌
os.listdir(dataset_folder_path)

['burglar', 'person']

In [60]:
#glob.glob(dataset_folder_path+'/*/*')
#파일의 경로+/*을 입력하면 해당 폴더 하위 경로 상에 있는 모든 파일을 리스트로 리턴
#dataset_folder_path+'/*/*' -> 하위 폴더의 하위 데이터 전체 리턴
for label, folder in enumerate(os.listdir(dataset_folder_path)):
  #print(dataset_folder_path + '/' + folder) 좋은 코드 아님
  #windows에는 \를 쓰므로 에러 #어떤 os에서든 돌아가게 만듬
  #print(os.path.join(dataset_folder_path, folder)) # 유진 보수 좋은 코드
  folder_full_path = os.path.join(dataset_folder_path, folder)
  all_files = glob.glob(folder_full_path + '/*.jpeg')
  all_files_png = glob.glob(folder_full_path + '/*.png')
  all_files_jpg = glob.glob(folder_full_path + '/*.jpg')
  all_files.extend(all_files_png)
  all_files.extend(all_files_jpg)
  #print(folder_full_path)
  #print(all_files)

  #labeling
  for idx, file_full_path in enumerate(all_files):
    #print(idx, file_full_path)
    image = PIL.Image.open(file_full_path)
    image = image.resize((64, 64))
    image = image.convert('RGB')
    #넘파이 행렬로 바꿈
    data = np.asarray(image)
    x.append(data)#x랑 y 현재 리슽 안에 넘파이 형태 있는 형태
    y.append(label)
    if idx % 30 == 0:
      print(idx, '/', len(all_files))



0 / 392
30 / 392
60 / 392
90 / 392
120 / 392
150 / 392
180 / 392
210 / 392
240 / 392
270 / 392
300 / 392
330 / 392
360 / 392
390 / 392
0 / 1001
30 / 1001
60 / 1001
90 / 1001
120 / 1001
150 / 1001
180 / 1001
210 / 1001
240 / 1001
270 / 1001
300 / 1001
330 / 1001
360 / 1001
390 / 1001
420 / 1001
450 / 1001
480 / 1001
510 / 1001
540 / 1001
570 / 1001
600 / 1001
630 / 1001
660 / 1001
690 / 1001
720 / 1001
750 / 1001
780 / 1001
810 / 1001
840 / 1001
870 / 1001
900 / 1001
930 / 1001
960 / 1001
990 / 1001


In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
from sklearn.model_selection import train_test_split
x = np.array(x)
y = np.array(y)
print('카테고리 개수', np.bincount(y))
print('전처리 확인', x.shape)

카테고리 개수 [ 392 1001]
전처리 확인 (1393, 64, 64, 3)


In [63]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [64]:
#전처리 시간이 너무 오래 걸리면(데이터 많을때)
#xy = (x_train, x_test, y_train, y_test)
#np.save('/content/drive/MyDrive/preprocessing_data.npy', xy)


In [65]:
#x_train, x_test, y_train, y_test = np.load('/content/drive/MyDrive/preprocessing_data.npy', allow_pickle=True)

In [66]:
x_train = x_train / 255.
x_test = x_test / 255.

In [67]:
#구글넷 실습
#레즈넷 실습 -> fine tuning

In [68]:
#개선의 여지 해상도 수정 (전처리도 수정 해야함) ex) 64X64 -> 100X100
input_layer = layers.Input(shape = (64, 64, 3), name='start_layer')

#개선의 여지 conv2d 수정(성능이 매우 잘 나온다면 - 모델 다이어트,
#성능이 안나온다면 channel 수, 레이어 추가)
tower_1 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)

tower_2 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)
tower_2 = layers.Conv2D(64, kernel_size = (3,3) , padding='same' , activation= 'relu')(tower_2)

tower_3 = layers.Conv2D(64, kernel_size = (1,1) , padding='same' , activation= 'relu')(input_layer)
tower_3 = layers.Conv2D(64, kernel_size = (5,5) , padding='same' , activation= 'relu')(tower_3)

tower_4 = layers.MaxPool2D(pool_size=(3,3), strides=(1,1) , padding='same')(input_layer)
tower_4 = layers.Conv2D(64, kernel_size=(1,1) , padding='same' , activation='relu')(tower_4)

concat_layer = layers.concatenate([tower_1, tower_2,tower_3,tower_4], axis=3)

tower_2_1 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)

tower_2_2 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)
tower_2_2 = layers.Conv2D(120, kernel_size = (3,3) , padding='same' , activation= 'relu')(tower_2_2)

tower_2_3 = layers.Conv2D(120, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer)
tower_2_3 = layers.Conv2D(120, kernel_size = (5,5) , padding='same' , activation= 'relu')(tower_2_3)

tower_2_4 = layers.MaxPool2D(pool_size=(3,3), strides=(1,1) , padding='same')(concat_layer)
tower_2_4 = layers.Conv2D(120, kernel_size=(1,1) , padding='same' , activation='relu')(tower_2_4)

concat_layer2 = layers.concatenate([tower_2_1, tower_2_2,tower_2_3,tower_2_4], axis=3)

tower_3_1 = layers.Conv2D(256, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer2)

tower_3_2 = layers.Conv2D(256, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer2)
tower_3_2 = layers.Conv2D(256, kernel_size = (3,3) , padding='same' , activation= 'relu')(tower_3_2)

tower_3_3 = layers.Conv2D(256, kernel_size = (1,1) , padding='same' , activation= 'relu')(concat_layer2)
tower_3_3 = layers.Conv2D(256, kernel_size = (5,5) , padding='same' , activation= 'relu')(tower_3_3)

tower_3_4 = layers.MaxPool2D(pool_size=(3,3), strides=(1,1) , padding='same')(concat_layer2)
tower_3_4 = layers.Conv2D(256, kernel_size=(1,1) , padding='same' , activation='relu')(tower_3_4)

final_concat_layer = layers.concatenate([tower_3_1, tower_3_2,tower_3_3,tower_3_4], axis=3)
average_pooling_layer = layers.AveragePooling2D(pool_size=(16,16),strides=(16,16))(final_concat_layer)
flat_layer = layers.Flatten()(average_pooling_layer)

#개선의 여지 - Dense 1층 추가
output_layer = layers.Dense(1 , activation = 'sigmoid')(flat_layer)

google_net_model = models.Model(inputs = input_layer, outputs = output_layer)
google_net_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 start_layer (InputLayer)    [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 conv2d_31 (Conv2D)          (None, 64, 64, 64)           256       ['start_layer[0][0]']         
                                                                                                  
 conv2d_33 (Conv2D)          (None, 64, 64, 64)           256       ['start_layer[0][0]']         
                                                                                                  
 max_pooling2d_5 (MaxPoolin  (None, 64, 64, 3)            0         ['start_layer[0][0]']         
 g2D)                                                                                       

In [73]:
#개선 여지 hyper parameter tuning
# ex) tf.keras.optimizers.Adam()
tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)
google_net_model.compile(loss='binary_crossentropy',
                         optimizer='Adam', metrics=['accuracy'])

In [74]:
check_point_path = '/content/drive/MyDrive/cycle_gan_checkpoint'
mc = tf.keras.callbacks.ModelCheckpoint(check_point_path,
                                   monitor='val_loss',
                                   save_best_only=True)

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                      patience=7)

In [75]:
google_net_model.fit(x_train, y_train, batch_size=32, epochs=30,
                     callbacks=[mc, es],
                     validation_data=(x_test, y_test))

Epoch 1/30
35/35 [==============================] - 27s 645ms/step - loss: 0.5984 - accuracy: 0.7074 - val_loss: 0.5800 - val_accuracy: 0.7133
Epoch 2/30
35/35 [==============================] - 20s 584ms/step - loss: 0.5583 - accuracy: 0.7226 - val_loss: 0.5152 - val_accuracy: 0.7240
Epoch 3/30
35/35 [==============================] - 20s 570ms/step - loss: 0.4945 - accuracy: 0.7415 - val_loss: 0.4508 - val_accuracy: 0.7563
Epoch 4/30
35/35 [==============================] - 20s 578ms/step - loss: 0.4131 - accuracy: 0.8142 - val_loss: 0.3466 - val_accuracy: 0.8495
Epoch 5/30
35/35 [==============================] - 20s 571ms/step - loss: 0.3836 - accuracy: 0.8276 - val_loss: 0.3320 - val_accuracy: 0.8459
Epoch 6/30
35/35 [==============================] - 21s 616ms/step - loss: 0.3669 - accuracy: 0.8285 - val_loss: 0.2950 - val_accuracy: 0.8530
Epoch 7/30
35/35 [==============================] - 17s 487ms/step - loss: 0.3202 - accuracy: 0.8618 - val_loss: 0.2969 - val_accuracy: 0.8710

In [76]:
##our model inference code
#inference만 진행할 시, 가장 위 import library 실행 -> 모델 빌드하는 부분 실행
google_net_model.load_weights('/content/drive/MyDrive/cycle_gan_checkpoint')

In [77]:
from google.colab import files
f = files.upload()

Saving waldo2-065.jpeg to waldo2-065.jpeg


In [78]:
file_name = list(f.keys())[0]
test_img = PIL.Image.open(file_name)
test_img = test_img.resize((64, 64))
test_img = test_img.convert('RGB')
#넘파이 행렬로 바꿈
data = np.asarray(test_img)
test_image_tensor = np.array([data])#x랑 y 현재 리슽 안에 넘파이 형태 있는 형태
test_image_tensor = test_image_tensor / 255.
test_image_tensor.shape

(1, 64, 64, 3)

In [79]:
result = google_net_model.predict(test_image_tensor)
result[0][0]
if result[0][0] > 0.5:
  print(f'{result[0][0] * 100} % 확률로 just person')
else:
  print(f'{(1-result[0][0]) * 100} % 확률로 is waldo')

1/1 [==============================] - 0s 181ms/step
99.99703512421547 % 확률로 is waldo
